In [ ]:
CALL db.schema.visualization()

CALL db.schema.nodeTypeProperties()

CALL db.schema.relTypeProperties()

SHOW CONSTRAINTS

##Find all :Person nodes with the name, Tom Hanks.

We then traverse the :ACTED_IN relationships to all :Movie nodes and filter for movies with a year property equal to 2013.

Return the movie titles.

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
AND m.year = 2013
RETURN m.title

##Testing Inequality
This query returns the names of all actors that acted in the movie Captain Phillips where Tom Hanks is excluded.

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name <> 'Tom Hanks'
AND m.title = 'Captain Phillips'
RETURN p.name

##Testing less than or greater than
You can test both numbers and strings for values less than (<) or greater than (>) a value. Adding the equals sign will include the specified number within the predicate.

In [ ]:
MATCH (m:Movie) WHERE m.title = 'Toy Story'
RETURN
    m.year < 1995 AS lessThan, //  Less than (false)
    m.year <= 1995 AS lessThanOrEqual, // Less than or equal(true)
    m.year > 1995 AS moreThan, // More than (false)
    m.year >= 1995 AS moreThanOrEqual // More than or equal (true)

##Testing Ranges
To test for property values within a range, you can use a combination of less than and greater than.

This query returns the four movies that Tom Hanks acted in between 2005 and 2010, inclusive.

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
AND  2005 <= m.year <= 2010
RETURN m.title, m.released

We can also use **OR** to expand the filtering to return more data as follows:

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
OR m.title = 'Captain Phillips'
RETURN p.name, m.title

##Testing null property values
A property for a node or relationship is null if it does not exist. You can test the existence of a property for a node using the IS NOT NULL predicate.

In [ ]:
MATCH (p:Person)
WHERE p.died IS NOT NULL
AND p.born.year >= 1985
RETURN p.name, p.born, p.died

And we can test if a property exists using the **IS NULL** predicate:

In [ ]:
MATCH (p:Person)
WHERE p.died IS NULL
AND p.born.year <= 1922
RETURN p.name, p.born, p.died

##Testing labels or patterns?
Depending on your data model, it may be useful to test that a node has a label. This is particularly **useful when a node may have multiple labels.**

You can test for **a label’s existence** on a node using the **{alias}:{label}** syntax.

In [ ]:
MATCH (p:Person)
WHERE  p.born.year > 1960
AND p:Actor
AND p:Director
RETURN p.name, p.born, labels(p)

#This query will retrieve all Person nodes with the label Actor and Director that were born after 1960. The labels() function returns the list of labels for a node.

Here is a variation of the previous query. Rather than using the Actor or Director labels, it uses the relationship types :ACTED_IN and :DIRECTED to imply that the node at the other end of the relationship has the correct label:


This query retrieves all people born after 1960 (assigned to the alias p), who also acted in a movie and directed the same movie. This query is more specific in that the same person both directed and acted in the movie. In the previous query, we were only looking at labels and not relationships to movies.

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(p)
WHERE  p.born.year > 1960
RETURN p.name, p.born, labels(p), m.title

##Discovering relationship types
A query with a pattern need not specify the relationship type in the query:

This query retrieves all Movie nodes that are related to Tom Hanks. Each row returned is a movie title and the type of relationship that Tom Hanks has to that movie. We use the type() function to return the type of the relationship, r. Notice that for this query, Tom Hanks has both an ACTED_IN and DIRECTED relationship to the movie, Larry Crowne.

In [ ]:
MATCH (p:Person)-[r]->(m:Movie)
WHERE  p.name = 'Tom Hanks'
RETURN m.title AS movie, type(r) AS relationshipType

##Testing list inclusion
You can test if a value is in a list property:

This query returns the titles, languages, countries of all movies that have Israel in their list of countries.

In [ ]:
MATCH (m:Movie)
WHERE "Israel" IN m.countries
RETURN m.title, m.languages, m.countries

##**Horror Movie Directors**
Write and execute a query to return the names of directors of horror
movies released in the year 2000. Use the year property of the Movie
 node to do the comparison as well as the name of "Horror" for the Genre node.

In [ ]:

MATCH (p:Director)-[:DIRECTED]->(m:Movie)
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)
WHERE m.year = 2000 AND g.name="Horror"
RETURN COUNT(p.name)

##History of all commands

In [ ]:
:history

###Write and execute a query to return all Movie nodes in the graph that do not have a tmdbId property.

How many Movie nodes are returned?

In [ ]:
MATCH (m:Movie)
WHERE
m.tmdbId IS NULL
RETURN COUNT(m)

##How many movies have no poster?
Write and execute a query to return the titles of all movies that do not have a poster.

In [ ]:
MATCH (m:Movie)
WHERE
m.poster IS NULL
RETURN COUNT(m.title)

##Find People Born in the Fifties
Using the sandbox on the right, write and execute a query to return people born in the 1950’s (1950 - 1959) that are both Actors and Directors.

How many Person nodes are returned?

In [ ]:
MATCH (p:Person)
WHERE 1950 <= p.born.year <= 1959
AND p:Actor
AND p:Director
RETURN COUNT(p.name)

##Finding German Language Movies
Write and execute a query to return movies released in the "German" language.

Note: You only need to search for "German"; there are elements in the database that have " German" as a language. You do not need to count these.

How many Movie node names are returned?

In our data model, every :Movie node has a languages property which contains an array of strings. For example ['English', 'German', 'Italian'].

In [ ]:
MATCH (m:Movie)
WHERE "German" IN m.languages
RETURN COUNT(m.title)

###When the property is a string type, you can filter by the starting characters in the string:

In [ ]:
MATCH (m:Movie)
WHERE  m.title STARTS WITH 'Toy Story'
RETURN m.title, m.released

###And you can filter queries whose properties end with a set of characters:

In [ ]:
MATCH (m:Movie)
WHERE  m.title ENDS WITH ' I'
RETURN m.title, m.released

###Additionally, you can test if a substring is contained in a property:

In [ ]:
MATCH (m:Movie)
WHERE  m.title CONTAINS 'River'
RETURN m.title, m.released

##Case-sensitive strings
String matching in Cypher is case-sensitive. If string values could be mis-interpreted if the letters do not match in case, your queries may miss data in the graph.

In [ ]:
MATCH (p:Person)
WHERE toLower(p.name) ENDS WITH 'demille'
RETURN p.name

###Conversely, we can do the same test with upper-case:

In [ ]:
MATCH (p:Person)
WHERE toUpper(p.name) ENDS WITH 'DEMILLE'
RETURN p.name

###Here is an example using CONTAINS:

In this query, we find all people that have **" de", " De", or " DE"** in their names.

In [ ]:
MATCH (p:Person)
WHERE toUpper(p.name) CONTAINS ' DE '
RETURN p.name

##About indexes for queries
If you transform a string property during a query, such as toUpper() or toLower(), the query engine turns off the use of the index.

With any query, you can always check if an index will be used by prefixing the query with **EXPLAIN.**

This query produces the execution plan where the first step is NodeIndexSeekByRange. In this case an index will be used because it is defined in the graph.

In [ ]:
EXPLAIN MATCH (m:Movie)
WHERE  m.title STARTS WITH 'Toy Story'
RETURN m.title, m.released

###But with this query the index will not be used, even if present in the graph, **because toLower() is used:**

In [ ]:
EXPLAIN MATCH (p:Person)
WHERE toLower(p.name) ENDS WITH 'demille'
RETURN p.name

###We want to find all actors whose name begins with Robert. What clause do you use?

In [ ]:
MATCH (p:Person)
WHERE p.name STARTS WITH 'Robert'
RETURN p.name

###What Cypher keyword can you use to determine if an index will be used for a query?

In [ ]:
 EXPLAIN

###Write and execute a query to return all Movie titles in the graph that have a title that begins with "Life is". There may be titles that do not adhere to capitalization as such so you must ensure that all titles will match. That is, it will retrieve any case of the string, such as "Life is", "LIFE IS", "life is", "Life Is".

How many Movie nodes are returned?

In [ ]:
MATCH (m:Movie)
WHERE toLower(m.title) STARTS WITH 'life is'
RETURN COUNT(m.title)

#OR

MATCH (m:Movie)
WHERE toUpper(m.title) STARTS WITH 'LIFE IS'
RETURN COUNT(m.title)

###Roles containing dog?
Write and execute a query to return the name of the person, their role, and the movie title where the role played by the actors or director had a value that included 'dog' (case-insensitive)? That is, the role could contain "Dog", "dog", or even "DOG".

In [ ]:
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE  toLower(r.role) CONTAINS "dog"
RETURN p.name, r.role, m.title

#OR

MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE  toUpper(r.role) CONTAINS "DOG"
RETURN p.name, r.role, m.title

#OR

MATCH (p:Person)-[r:DIRECTED]->(m:Movie)
WHERE  toUpper(r.role) CONTAINS "DOG"
RETURN p.name, r.role, m.title

#OR

MATCH (p:Person)-[r:DIRECTED]->(m:Movie)
WHERE  toLower(r.role) CONTAINS "dog"
RETURN p.name, r.role, m.title

#OR

MATCH (p:Person)-[r:DIRECTED]->(m:Movie)
WHERE  p.Actor OR p.Director OR toLower(r.role) CONTAINS "dog"
RETURN COUNT(r.role)

##Patterns in the graph
The Neo4j graph engine is implemented to traverse relationships very quickly. As you become more experienced with Cypher queries, you will soon learn that there are multiple ways to write a query that returns the same results. The difference in queries is typically its traversal performance. In this lesson, you will begin learning about graph traversal and query performance.

A pattern is a **combination of nodes and relationships that is used to traverse the graph at runtime.**


This query:

1. Retrieves the anchor of the query, the Tom Hanks :Person node.

2. It then follows the :ACTED_IN relationship to a :Movie node.

3. Then, for the Movie node and Person node, it tests whether these nodes are related by the DIRECTED relationship.

If they are, then the row is returned.

This **exists { }** test is done for every Movie node related to Tom Hanks as an actor. This query returns the single movie that Tom Hanks directed and acted in.

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE  p.name = 'Tom Hanks'
AND exists {(p)-[:DIRECTED]->(m)}
RETURN p.name, labels(p), m.title

##Profiling queries
You can use the PROFILE keyword to show the **total number of rows retrieved from the graph in the query.**


In the profile, you can see that the initial row is retrieved, but then 38 rows are retrieved for each Movie that Tom Hanks acted in. Then the test is done for the :DIRECTED relationship.

In [ ]:
PROFILE MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE  p.name = 'Tom Hanks'
AND exists {(p)-[:DIRECTED]->(m)}
RETURN m.title


###This is a better way to do the same query. This is a query that you have seen before.

The query:

Retrieves the anchor (the Tom Hanks Person node).

It then finds a Movie node where Tom Hanks is related to with the ACTED_IN relationship.

It then traverses all DIRECTED relationships that point to the same Tom Hanks node.

This traversal is very efficient because the graph engine can take the [internal] relationship cardinalities into account. If you execute this query, it returns the same result as the previous query

Notice, however that this query is much more efficient. It retrieves one row then two rows; much less data than the first query. Note that the performance of queries that use patterns will depend upon the data model for your graph and also the number of nodes in the traversal.



In [ ]:
PROFILE MATCH (p:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(p)
WHERE  p.name = 'Tom Hanks'
RETURN  m.title

##EXPLAIN vs PROFILE

The difference between using EXPLAIN and PROFILE is that **EXPLAIN provides estimates of the query steps** where **PROFILE provides the exact steps and number of rows retrieved for the query.** Providing you are simply querying the graph and **not updating anything,** it is fine to execute the query multiple times using PROFILE. In fact, as part of query tuning, you should execute the query **at least twice as the first execution involves the generation of the execution plan which is then cached.** That is, the first PROFILE of a query will always be **more expensive than subsequent queries.**

####Query tuning is beyond the scope of this course, but it is important to profile your queries so that you can optimize the queries of your application. The metric that is typically a **good measure of query performance is the number db hits.**

##Finding non-patterns
You’ve just seen that using a pattern and testing the existence of a pattern in our graph for this course is not optimal. This may or may not be the case, depending on your data model. There is a scenario where using **exists { }** for a pattern is useful. You use **NOT exists { }** to exclude patterns in the graph.

We want to find all the movies that Tom Hanks **acted in, but did not direct.**

Here we want to exclude the :DIRECTED relationships to movies for Tom Hanks. If you profile this query, you will find that it is not performant, but it is the only way to perform this query.

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE  p.name = 'Tom Hanks'
AND NOT exists {(p)-[:DIRECTED]->(m)}
RETURN  m.title

##We want to return the movies that Clint Eastwood acted in and directed.

How would you complete this query?

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE  p.name = "Clint Eastwood"
AND exists { (p)-[:DIRECTED]→(m) }
RETURN  m.title

##Query performance
What Cypher keyword helps you to understand the performance of a query when it runs?

In [ ]:
PROFILE

##Write and execute a query to return the titles of all movies that Rob Reiner directed, but did not act in.

How many Movie titles are returned?

In [ ]:
MATCH (p:Person)-[:DIRECTED]->(m:Movie)
WHERE  p.name = 'Rob Reiner'
AND NOT exists {(p)-[:ACTED_IN]->(m)}
RETURN  COUNT(m.title)

##Profiling a query
What is the **total number of db hits** when this query executes?

In [ ]:
PROFILE MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Clint Eastwood'
AND NOT exists {(p)-[:DIRECTED]->(m)}
RETURN m.title

#Cypher version: 5, planner: COST, runtime: PIPELINED. 489 total db hits in 2 ms  <------seee db hits

##Using multiple MATCH clauses
Here is an example of a query that contains two explicit MATCH clauses:

This query:

Retrieves the anchor nodes (movies released after the year 2000) and the set of actors for each movie.

It then follows the :DIRECTED relationships to each Movie node to retrieve the director of each movie.

It returns the triple of actor name, movie title, director name.

**Notice that actors, movie titles, directors are repeated in the rows returned.**

In [ ]:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.year > 2000
MATCH (m)<-[:DIRECTED]-(d:Person)
RETURN a.name, m.title, d.name

##Using multiple patterns in the MATCH clause
An alternative to using multiple MATCH clauses is to specify multiple patterns:

In this query, multiple patterns are specified. In the second pattern, the variable m is used from the first pattern.

In general, using **a single MATCH clause will perform better than multiple MATCH clauses.** This is because **relationship uniqueness is enforced so there are fewer relationships traversed.**

In [ ]:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie),
      (m)<-[:DIRECTED]-(d:Person)
WHERE m.year > 2000
RETURN a.name, m.title, d.name

##Using a single pattern
And finally, the **same query can be written** as follows:

In most cases, specifying a **single pattern** will yield the best performance.

The **performance of your queries** will depend greatly upon your **data model and the size of the graph.**

In [ ]:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(d:Person)
WHERE m.year > 2000
RETURN a.name, m.title, d.name

##Optionally matching rows
In this query:

1. We find the movie node for Kiss Me Deadly.

2. Then we find all movies, rec that are in the same genre as Kiss Me Deadly.

3. Then we find the actors that acted in both rec and Kiss Me Deadly.

In [ ]:
MATCH (m:Movie) WHERE m.title = "Kiss Me Deadly"
MATCH (m)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)
MATCH (m)<-[:ACTED_IN]-(a:Actor)-[:ACTED_IN]->(rec)
RETURN rec.title, a.name

##OPTIONAL MATCH
OPTIONAL MATCH matches patterns with your graph, just like MATCH does. The difference is that **if no matches are found, OPTIONAL MATCH will use nulls for missing parts of the pattern.** OPTIONAL MATCH could be considered the Cypher equivalent of the **outer join in SQL.**


This query returns rows where the pattern where an actor acted in both movies is optional and a null value is returned for any row that has no value. In general, and depending on your graph, an optional match will return more rows.

In [ ]:
MATCH (m:Movie) WHERE m.title = "Kiss Me Deadly"
MATCH (m)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)
OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Actor)-[:ACTED_IN]->(rec)
RETURN rec.title, a.name

##1. Optionally matching a pattern
We want to return the names of all actors whose name begins with Tom and also the title of the movies they directed. If they did not direct the movie, then return a null value.

How would you complete this query?

In [ ]:
MATCH (p:Person)
WHERE p.name STARTS WITH 'Tom'
OPTIONAL MATCH (p)-[:DIRECTED]→(m:Movie)
RETURN p.name, m.title

##2. Optional matches
What value does OPTIONAL MATCH return if there is no value for a string property being returned in a row?

In [ ]:
NULL

###Retrieving movies in a genre with their reviewers
Here is a query that returns the titles of all Film Noir movies and the users who rated them.

In [ ]:
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)
WHERE g.name = 'Film-Noir'
MATCH (m)<-[:RATED]-(u:User)
RETURN m.title, u.name

##Ordering results
Whether you return results as **nodes or as properties of nodes,** you can specify a property value for the ordering.

**Strings** are ordered by their **text values.**

**Boolean true comes before false** when ordered.

**Numeric data (including date and datetime properties)** are ordered by their **numeric value.**

You specify **ORDER BY** in the RETURN clause where you specify the property for the ordering.


In this query, we find all Person nodes with a born year of 1980. The born property of Person nodes is a Cypher Date type, so we can use the year method to test the year value. Notice that is this code we are specifying aliases for the column headers for each row. The rows will be ordered by the value of the born property.

In [ ]:
MATCH (p:Person)
WHERE p.born.year = 1980
RETURN p.name AS name,
p.born AS birthDate
ORDER BY p.born

####The **default ordering is ascending,** but you can specify descending as follows:

In [ ]:
MATCH (p:Person)
WHERE p.born.year = 1980
RETURN p.name AS name, p.born AS birthDate
ORDER BY p.born DESC

##Eliminating null values returned
For some queries, you may want to **not return null values,** especially if you are ordering the data.

In [ ]:
MATCH (p:Person)
RETURN p.name AS name, p.born AS birthDate
ORDER BY p.born DESC

####This query returns many rows at the beginning of result set that have a null value for the born property.

In this case, we don’t want those nodes to be retrieved. We adjust the query as follows:

In [ ]:
MATCH (p:Person)
WHERE p.born IS NOT NULL
RETURN p.name AS name, p.born AS birthDate
ORDER BY p.born DESC

##Ordering multiple results
You can provide multiple sort expressions and the result will be sorted in that order.

In this query we **return the movies in descending year order** and **then the movie titles in ascending order.**

There is **no limit to the number of properties** you can order by.

In [ ]:
MATCH (p:Person)-[:DIRECTED | ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
OR p.name = 'Keanu Reeves'
RETURN  m.year, m.title
ORDER BY m.year DESC , m.title

###1. Ordering result rows
We want to return the ratings that Sandy Jones gave movies and return the rating from highest to lowest. What code do you use?

In [ ]:
MATCH (u:User)-[r:RATED]->(m:Movie)
WHERE u.name = 'Sandy Jones'
RETURN m.title AS movie, r.rating AS rating ORDER BY r.rating DESC


###2. Ordering results
What is the maximum number of properties can you order in your results?

In [ ]:
Unlimited

###Movies with ratings
Write and execute a query to return the movie titles where they are ordered from the highest to the lowest imdbRating value. In your query, only return movies that have a value for the imdbRating property.

How many Movie titles are returned?

In [ ]:
MATCH (m:Movie)
WHERE m.imdbRating IS NOT NULL
RETURN m.title, m.imdbRating
ORDER BY m.imdbRating DESC

##Highest rating
When you execute the previous query:

In [ ]:
MATCH (m:Movie)
WHERE m.imdbRating IS NOT NULL
RETURN m.title, m.imdbRating
ORDER BY m.imdbRating DESC

###Youngest actor in highest rated movie
Modify the query you just executed to:

1. Match the people that acted in the movies.

2. Return the names of people sorted by the born property

The question you want to answer is:

What is the youngest actor that acted in the most highly-rated movie?

Enter the name (case-sensitive!):

In [ ]:
MATCH (m:Movie)<-[ACTED_IN]-(p:Person)
WHERE m.imdbRating IS NOT NULL
RETURN m.title, m.imdbRating, p.name, p.born
ORDER BY m.imdbRating DESC, p.born DESC

##Limiting results
Although you can filter queries to reduce the number of results returned, you may also want to limit the number of results returned. This is useful if you have very large result sets and you only need to see the beginning or end of a set of ordered results. You can use the LIMIT keyword to specify the number of results returned.

In [ ]:
MATCH (m:Movie)
WHERE m.released IS NOT NULL
RETURN m.title AS title,
m.released AS releaseDate
ORDER BY m.released DESC LIMIT 100

###Or, we may want to determine the youngest person in the graph:

In [ ]:
MATCH (p:Person) WHERE
p.born IS NOT NULL
RETURN p.name as name,
p.born AS birthDate
ORDER BY p.born DESC LIMIT 1

###Skipping some results
In an ordered result set, you may want to control what results are returned. This is useful in an application where pagination is required.

In this query we are returning the names of people born in 1980 ordered by their birth date.

In [ ]:
MATCH (p:Person)
WHERE p.born.year = 1980
RETURN  p.name as name,
p.born AS birthDate
ORDER BY p.born

####You can add a SKIP and LIMIT keyword to control what page of results are returned.

In this query, we return 10 rows representing page 5, where each page contains 10 rows.

In [ ]:
MATCH (p:Person)
WHERE p.born.year = 1980
RETURN  p.name as name,
p.born AS birthDate
ORDER BY p.born SKIP 40 LIMIT 10

##Eliminating duplicate records
You have seen a number of query results where there is duplication in the results returned. In some cases, you may want to eliminate duplicated results. You do so by using the **DISTINCT** keyword.

Tom Hanks both acted in and directed the movie, Larry Crowne, so the movie is returned twice in the result stream.

We can eliminate the duplication by specifying the DISTINCT keyword as follows:

In [ ]:
MATCH (p:Person)-[:DIRECTED | ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
RETURN m.title, m.released
ORDER BY m.title

####We can eliminate the duplication by specifying the DISTINCT keyword as follows:

In [ ]:
MATCH (p:Person)-[:DIRECTED | ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
RETURN DISTINCT m.title, m.released
ORDER BY m.title

##Uses of DISTINCT
You can use DISTINCT to eliminate duplication of:

1. rows returned (you have just learned this)

2. property values

3. nodes

Here is an example where we **eliminate duplicate property values:**

In the above query, only a single value will be returned for each Movie year. If you were to not use DISTINCT, all Movie year values would be returned.

In [ ]:
MATCH (m:Movie)
RETURN DISTINCT m.year
ORDER BY m.year

####And here is an example where we eliminate duplicate nodes:



If we do not specify DISTINCT in the above query, the query returns a duplicate movie node.



In [ ]:
MATCH (p:Person)-[:DIRECTED | ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
RETURN  DISTINCT m

##1. What movies have reviews?
We want to return the movies that have been reviewed.

How would you complete this query so that duplicate movie titles are not returned?



In [ ]:
MATCH (m:Movie)<-[:RATED]-()
RETURN DISTINCT m.title

##2. Reducing data returned
Why would you want to use LIMIT in a RETURN clause?

In [ ]:
To reduce the amount of data returned to the client.

##Lowest rated movie
A movie has a property, imdbRating. Not all movies have a value for this property. Write and execute a query to determine the lowest imdbRating that a movie has in our graph.

What is the lowest imdbRating?

In [ ]:
MATCH(m:Movie)
WHERE m.imdbRating IS NOT NULL
RETURN m.imdbRating
ORDER BY m.imdbRating ASC LIMIT 1

####Here is a query that returns the names people who acted or directed the movie Toy Story and then retrieves all people who acted in the same movie.

Execute this query. It will return 183 rows, some of which are duplicates.

In [ ]:
MATCH (p:Person)-[:ACTED_IN| DIRECTED]->(m:Movie)
WHERE m.title = 'Toy Story'
MATCH (p)-[:ACTED_IN]->()<-[:ACTED_IN]-(p2:Person)
RETURN  DISTINCT p.name, p2.name

##Map projections
Many applications that access Neo4j via their drivers use Cypher to retrieve data from the graph as objects that will be used by the application. In Neo4j Browser, when nodes are returned, you can either view them as a graph, or you can view them in table view where all properties for a node are a single row. The data is returned as rows of data where each row represents a JSON-style object for a node.


This query returns all Person nodes that contain the string "Thomas". If you view the data returned as a table, it returns internal node information such as labels and identity, along with the property values.

In [ ]:
MATCH (p:Person)
WHERE p.name CONTAINS "Thomas"
RETURN p AS person
ORDER BY p.name ASC

####Another way that you can return data is without the internal node information, that is, only property values.

{
  "tmdbId": "1198886",

  "name": " Thomas Haneke",

  "imdbId": "0359735",

  "url": "https://themoviedb.org/person/1198886"

}


This query returns an object named person that contains all of the property values for the node. It does **not contain any of the internal information for the node such as its labels or id.**

In [ ]:
MATCH (p:Person)
WHERE p.name CONTAINS "Thomas"
RETURN p { .* } AS person
ORDER BY p.name ASC

####Additionally, you can customize what properties you return in the objects.

{

  "born": null,

  "name": " Thomas Haneke"

}


Here the person objects returned will include the name and born properties.

Being able to customize how data is returned is useful, especially if you are using a programming language that can work with JSON-style objects.

In [ ]:
MATCH (p:Person)
WHERE p.name CONTAINS "Thomas"
RETURN p { .name, .born } AS person
ORDER BY p.name

####Here is an example, where we are **adding information to the objects returned that are not part of the data in the graph.**

{

  "budget": 6400000,

  "movieId": "6993",

  "tmdbId": "5143",

  "imdbVotes": 51032,

  "runtime": 103,

  **"favorite": true,**

  "countries": [

    "USA"

  ],

  "imdbId": "0091167",

  "url": "https://themoviedb.org/movie/5143",

  "plot": "Between two Thanksgivings two years apart, Hannah's husband falls in love with her sister Lee, while her hypochondriac ex-husband rekindles his relationship with her sister Holly.",

  "released": "1986-03-14",

  "languages": [

    "English"

  ],
  "imdbRating": 8.0,

  "title": "Hannah and Her Sisters",

  "poster": "https://image.tmdb.org/t/p/w440_and_h660_face/gARgIRb2QFRFVrsziwWE389u1pK.jpg",

  "year": 1986,

  "revenue": 40084041

}


In addition to returning all property values for each Woody Allen movie, we are returning a property of favorite with a value of true for each Movie object returned.

In [ ]:
MATCH (m:Movie)<-[:DIRECTED]-(d:Director)
WHERE d.name = 'Woody Allen'
RETURN m {.*, favorite: true} AS movie

##1. Returning selected properties
We want to return the title and release date as Movie objects for all Woody Allen movies. Select the correct RETURN clause to do this.

In [ ]:
MATCH (m:Movie)<-[:DIRECTED]-(d:Director)
WHERE d.name = 'Woody Allen'
RETURN m {.title, .released} AS movie
ORDER BY m.released

###2. Returning data

MATCH (p:Person)

WHERE p.name CONTAINS "Thomas"

RETURN p AS person ORDER BY p.name

What is returned in every row?

In [ ]:
labels, identity, elementId, type and properties.

##Changing data returned
You can always change the data that is returned by performing string or numeric operations on the data.

This query returns the actors in each Toy Story movie and their age if they are still living according to our graph. We **add data to each line by calculating the actor’s age.**

In [ ]:
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
WHERE m.title CONTAINS 'Toy Story' AND
p.died IS NULL
RETURN m.title AS movie,
p.name AS actor,
p.born AS dob,
date().year - p.born.year AS ageThisYear

###Here is an example where we concatenate string data returned:

In [ ]:
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
WHERE m.title CONTAINS 'Toy Story' AND
p.died IS NULL
RETURN 'Movie: ' + m.title AS movie,
p.name AS actor,
p.born AS dob,
date().year - p.born.year AS ageThisYear

###Conditionally changing data returned
Cypher has a **CASE** clause that you can specify to compute the data returned which may be different from what is in the graph.

In [ ]:
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
WHERE p.name = 'Henry Fonda'
RETURN m.title AS movie,
CASE
WHEN m.year < 1940 THEN 'oldies'
WHEN 1940 <= m.year < 1950 THEN 'forties'
WHEN 1950 <= m.year < 1960 THEN 'fifties'
WHEN 1960 <= m.year < 1970 THEN 'sixties'
WHEN 1970 <= m.year < 1980 THEN 'seventies'
WHEN 1980 <= m.year < 1990 THEN 'eighties'
WHEN 1990 <= m.year < 2000 THEN 'nineties'
ELSE  'two-thousands'
END
AS timeFrame

##1. What age?
We want to return information about actors who acted in the Toy Story movies. We want to return the age that an actor will turn this year or that the actor died.

How would you complete this query?

In [ ]:
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
WHERE m.title CONTAINS 'Toy Story'
RETURN m.title AS movie,
p.name AS actor,
p.born AS dob,
CASE
WHEN p.died IS NULL THEN date().year - p.born.year
WHEN p.died IS NOT NULL THEN "Died"
END
AS ageThisYear

###2. Keywords used for conditionally returning values
What keywords can you use in a RETURN clause to conditionally return a value?

In [ ]:
CASE, WHEN, ELSE ,END

###Conditionally returning values for rows
Here is a query that returns the movies that Charlie Chaplin has acted in and the runtime for the movie.

Modify this query to return "Short" for runTime if the movie’s runtime is < 120 (minutes) and "Long" for runTime if the movie’s runtime is >= 120.

In [ ]:
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
WHERE p.name = 'Charlie Chaplin'
RETURN m.title AS Movie,
CASE
WHEN m.runtime < 120 THEN "Short"
WHEN m.runtime >= 120 THEN "Long"
ELSE "Unknown"
END AS Runtime

##Using count() to aggregate data
Cypher has a **count() function that you can use to perform a count of nodes, relationships, paths, rows during query processing.** When you aggregate in a Cypher statement, this means that the query must process all patterns in the MATCH clause to complete the aggregation to either return results or perform the next part of the query.

This query returns the number of movies Tom Hanks acted in.



In [ ]:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
WHERE a.name = 'Tom Hanks'
RETURN a.name AS actorName,
count(*) AS numMovies

####In this query we can see that actors and directors worked together in the same movie and how many collaborations there were.

In [ ]:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(d:Person)
RETURN a.name AS actorName,
d.name AS directorName,
count(*) AS numMovies
ORDER BY numMovies DESC

##Eager aggregation
When you aggregate in a Cypher statement, the query must process all patterns in the **MATCH** clause to complete the aggregation to return results. This is called eager aggregation.

The query engine processes all nodes and relationships in the pattern so that it can perform a count of all movies for a particular actor/director pair in the graph. Then, it returns the results grouped by the name of the director and actor pair.

For this query:

Clint Eastwood acted in and directed 20 movies.

Woody Allen acted in and directed 19 movies.

Adam Sandler acted in the same 8 movies that Dennis Dugan directed.

a.name

d.name

numMovies

--------------------------------
"Clint Eastwood"

"Clint Eastwood"

20

----------------------------------

"Woody Allen"

"Woody Allen"

19

--------------------------------------------------------


In [ ]:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(d:Person)
RETURN a.name, d.name,
count(*) AS numMovies
ORDER BY numMovies DESC

Aggregation in Cypher is different from aggregation in SQL. In Cypher, you need not specify a grouping key. As soon as an aggregation function like count() is used, all non-aggregated result columns become grouping keys.

The grouping is implicitly done, based upon the fields in the RETURN clause.

If you specify count(n), the graph engine calculates the number of non-null occurrences of n. If you specify count(*), the graph engine calculates the number of rows retrieved, including those with null values.

##Lists in the graph
You have already seen data in the graph that is stored as lists:

For this query, notice that these lists are shown in square brackets and each element of the list is separated by a comma. These lists contain a set of strings.

------------------------------
languages

countries

----------------------------------

["English"]

["USA"]

-----------------------------------

["English", " French"]

["USA"]

-------------------------------------------------

In [ ]:
MATCH (m:Movie)
RETURN m.languages AS languages,
m.countries AS countries
LIMIT 10

##Returning a list
You can return a list by specifying the square brackets:

This query returns the name, and the lifeTime list that includes the born and died values for each person.

p.name

lifeTime

-------------------------

"François Lallement"

[
"1877-02-04"
,
"1954-01-01"
]

----------------------------------

In [ ]:
MATCH (p:Person)
RETURN p.name, [p.born, p.died] AS lifeTime
LIMIT 10

##Using collect() to create a list
Cypher has a built-in aggregation function, collect() that enables you to aggregate values into a list. The value can be any expression, for instance a property value, a node, or result of a function or operation.


This query returns a list of movie titles associated with each actor. The rows are presented such that actors with the greatest number of titles are returned first.

In [ ]:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
RETURN a.name AS actor,
count(*) AS total,
collect(m.title) AS movies
ORDER BY total DESC LIMIT 10

##Eliminating duplication in lists
Just as you have learned to eliminate duplication in data returned using DISTINCT, you can also eliminate duplication in a list.

Here is a query that has duplication in its result:

When this query executes, it returns duplicate titles in the Movies list.

In [ ]:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.year = 1920
RETURN  collect(m.title) AS movies,
collect( a.name) AS actors

You can eliminate duplication in the list as follows:

In [ ]:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.year = 1920
RETURN  collect( DISTINCT m.title) AS movies,
collect( a.name) AS actors

##Collecting nodes
Rather than collecting the values of the title properties for movies, you can collect the nodes. For this simple query, it is the same visual result as returning m, but for more complex queries, you will learn later in this course that collecting nodes and using them for a later step of the query is useful.

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name ='Tom Cruise'
RETURN collect(m) AS tomCruiseMovies

##Accessing elements of a list
You can access particular elements of the list using the [index-value] notation where a list begins with index 0.

In this example we return the first cast member for each movie.

--------------------------

movie

castMember

castSize

----------------------------

"Toy Story"

"Jim Varney"

4

------------------------------

In [ ]:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
RETURN m.title AS movie,
collect(a.name)[0] AS castMember,
size(collect(a.name)) as castSize

You can also return a slice of a collection.

This query returns the second to the end of the list names of actors.

In [ ]:
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
RETURN m.title AS movie,
collect(a.name)[2..] AS castMember,
size(collect(a.name)) as castSize

##Other aggregating functions
There are more aggregating functions that may be useful for your application such as:

min()

max()

avg()

stddev()

sum()

##count() versus size()
You can either use count() to count the number of rows, or alternatively, you can return the size of the collected results. The size() function returns the number of elements in a list.

Here we use size() to return the number of elements in the list:

-------------------------

actor.name

director.name

collaborations

movies
-------------------------------------

"Jim Varney"

"John Lasseter"

1

["Toy Story"]

----------------------------------------

As you gain more experience with query tuning, you will learn that **count() may be more efficient** because it gets its values for node counts or relationships from a node from the internal count store of the graph.

In [ ]:
MATCH (actor:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(director:Person)
RETURN actor.name, director.name,
size(collect(m)) AS collaborations,
collect(m.title) AS movies

If we profile this query (run it at least twice):



In [ ]:
//Query 1
PROFILE MATCH (g:Genre)<-[:IN_GENRE]-(m)
RETURN g.name AS genre,
size(collect(m)) AS numMovies
ORDER BY size(collect(m)) DESC

#Cypher version: 5, planner: COST, runtime: PIPELINED. 20478 total db hits in 227 ms

This query uses count():

The Query 1 has different performance characteristics. You should always test the performance of your queries before you deploy your application.

In [ ]:
//Query 2
PROFILE MATCH (g:Genre)<-[:IN_GENRE]-(m:Movie)
RETURN g.name AS genre,
count(m) AS numMovies
ORDER BY count(m) DESC

#Cypher version: 5, planner: COST, runtime: PIPELINED. 61158 total db hits in 80 ms

####What genre of movies is the highest and how many movies are in that genre?

In [ ]:
MATCH (g:Genre)<-[:IN_GENRE]-(m:Movie)
RETURN g.name AS genre,
count(m) AS numMovies
ORDER BY count(m) DESC LIMIT 1

##List comprehension
You can create a **list by evaluating an expression** that tests for list inclusion.

In [ ]:
MATCH (m:Movie)
RETURN m.title as movie,
[x IN m.countries WHERE x CONTAINS 'USA' OR x CONTAINS 'Germany']
AS country LIMIT 500

##Pattern comprehension
Pattern comprehension is a very powerful way to create lists without changing the cardinality of the query. It behaves like an **OPTIONAL MATCH combined with collect().**

Notice that for pattern comprehension we specify the list with the square braces to include the pattern followed by the pipe character to then specify what value will be placed in the list from the pattern.

[

pattern

|

value

]

-----------------------------------

m.title

directors

actors

-----------------------

"Mission: Impossible - Rogue Nation"

["Christopher McQuarrie"]

["Simon Pegg", "Tom Cruise", "Jeremy Renner", "Rebecca Ferguson"]

--------------------------------

###This query returns the movies released in 2015. Each row contains the title of the movie, the list of director names, and the list of actor names.

In [ ]:
MATCH (m:Movie)
WHERE m.year = 2015
RETURN m.title,
[(dir:Person)-[:DIRECTED]->(m) | dir.name] AS directors,
[(actor:Person)-[:ACTED_IN]->(m) | actor.name] AS actors

###Using pattern comprehension to **create a list where we specify a filter for the pattern.**

-------------------------------

movies

------------------------------------

["Toy Story of Terror: 2013", "Toy Story 3: 2010", "Toy Story 2: 1999", "Toy Story: 1995"]

-----------------------------------------

For this pattern comprehension, the title of the movie is concatenated with the year of the movie as a value to add as an element of the list returned.

In [ ]:
MATCH (a:Person {name: 'Tom Hanks'})
RETURN [(a)-->(b:Movie)
WHERE b.title CONTAINS "Toy" | b.title + ": " + b.year]
AS movies

##Working with maps
A Cypher map is **list of key/value pairs** where each element of the list is of the format **'key': value.** A node or relationship can have a property that is a map.

For example, a map of months and the number of days per month could be:

**{Jan: 31, Feb: 28, Mar: 31, Apr: 30 , May: 31, Jun: 30 , Jul: 31, Aug: 31, Sep: 30, Oct: 31, Nov: 30, Dec: 31}**

Using this map, we can return the value for one of its elements:

Here we use the key, 'Feb' to access its value.

In [ ]:
RETURN {Jan: 31, Feb: 28, Mar: 31, Apr: 30 ,
May: 31, Jun: 30 , Jul: 31, Aug: 31, Sep: 30,
Oct: 31, Nov: 30, Dec: 31}['Feb'] AS daysInFeb

####Alternatively, you can access a value with the **'.' notation:**

In [ ]:
RETURN {Jan: 31, Feb: 28, Mar: 31, Apr: 30 ,
May: 31, Jun: 30 , Jul: 31, Aug: 31, Sep: 30,
Oct: 31, Nov: 30, Dec: 31}.Feb AS daysInFeb

####And you can return **a list of keys** of a map as follows:

In [ ]:
RETURN keys({Jan: 31, Feb: 28, Mar: 31, Apr: 30 ,
May: 31, Jun: 30 ,Jul: 31, Aug: 31, Sep: 30,
Oct: 31, Nov: 30, Dec: 31}) AS months

####A node in the graph, when returned in Neo4j Browser is **a map,** when displayed as table rows. For example, a Movie node:

In [ ]:
MATCH (m:Movie)
RETURN m LIMIT 1

##Map projections
Map projections are **when you can use retrieved nodes to create or return some of the information in the nodes.** A Movie node can have the properties title, released, and tagline. Suppose we want to return the Movie node information, but without the tagline property? You can do so using map projections:

In [ ]:
MATCH (m:Movie)
WHERE m.title CONTAINS 'Matrix'
RETURN m { .title, .released } AS movie

##1. List of actors in the movie Toy Story
We want to return the list of names of actors in the movie Toy Story as a single row. What code do you use?

In [ ]:
MATCH (movie:Movie {title:'Toy Story'})<-[:ACTED_IN]-(actor:Person)
RETURN collect(actor.name) AS actors

##2. Aggregating data
What Cypher function can you use to return the number of elements in a list of Movie nodes, movies?

In [ ]:
size(movies) is the correct answer.

size(collect(movies)) is incorrect because movies is already a list so you do not need to create another list.

count() is used to count the number of rows or variables returned, but is not used to return the number of elements in a list.

##Most active director?
Using count() write a query to return the number of movies a person directed.

What is the highest number of movies a director directed in our graph?

In [ ]:
MATCH (d:Director)-[:DIRECTED]-(m)
RETURN d.name AS Director,
count(*) AS numMovies
ORDER BY numMovies DESC
LIMIT 5

###Largest Number of Actors
Write and execute a query to return the list of actors for a movie with a given title. Order and limit the results so that the movie title with the largest number of actors is returned.

Hint: There are multiple movies with the same title so you are aggregating by the movie title and not the movieID.

What movie title had the largest number of actors? (Enter a case-sensitive string for the movie title.)

In [ ]:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
RETURN m.title AS movie,
collect(a.name) AS actors,
size(collect(a.name)) AS num
ORDER BY num DESC LIMIT 1

##How Many Actors?
Use the query you just wrote. For the largest cast, how many actors were in the movies with the title "Hamlet"?

In [ ]:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
WHERE m.title ="Hamlet"
RETURN count(a.name) AS numActors

##Date and Time data in Cypher
Cypher has these basic formats for storing date and time data.

In [ ]:
RETURN date(), datetime(), time()

##Create a node test with date properties.

In [ ]:
MERGE (x:Test {id: 1})
SET x.date = date(),
    x.datetime = datetime(),
    x.time = time()
RETURN x

In [ ]:
{
  "identity": 33880,
  "labels": [
    "Test"
  ],
  "properties": {
    "date": "2024-08-23",
    "datetime": "2024-08-23T18:22:26.511000000Z",
    "time": "18:22:26.511000000Z",
    "id": 1
  },
  "elementId": "33880"
}

Next, execute this code that will show the types of the properties in the graph:

In [ ]:
CALL apoc.meta.nodeTypeProperties()

###Extracting components of a date or datetime
You can access the components of a date or datetime property:

In [ ]:
MATCH (x:Test {id: 1})
RETURN x.date.day, x.date.year,
x.datetime.year, x.datetime.hour,
x.datetime.minute

##Setting date values
You can use a string to set a value for a date:

In [ ]:
MATCH (x:Test {id: 1})
SET x.date1 = date('2022-01-01'),
    x.date2 = date('2022-01-15')
RETURN x

###Setting datetime values
You can use a string to set a value for a datetime:

In [ ]:
MATCH (x:Test {id: 1})
SET x.datetime1 = datetime('2022-01-04T10:05:20'),
    x.datetime2 = datetime('2022-04-09T18:33:05')
RETURN x

##Working with durations
A duration is used to determine the difference between two date/datetime values or to add or subtract a duration to a value.

This code returns the duration between date1 and date2 in the graph:

In [ ]:
MATCH (x:Test {id: 1})
RETURN duration.between(x.date1,x.date2)

We can return the duration in days between two datetime values:

In [ ]:
MATCH (x:Test {id: 1})
RETURN duration.inDays(x.datetime1,x.datetime2).days

We can add a duration of 6 months:

In [ ]:
MATCH (x:Test {id: 1})
RETURN x.date1 + duration({months: 6})

###Using APOC to format dates and times
The APOC library has many useful functions for working with all types of data.

Here is one way you can use APOC to format a datetime:

In [ ]:
MATCH (x:Test {id: 1})
RETURN x.datetime as Datetime,
apoc.temporal.format( x.datetime, 'HH:mm:ss.SSSS')
AS formattedDateTime

Here is another example:



In [ ]:
MATCH (x:Test {id: 1})
RETURN apoc.date.toISO8601(x.datetime.epochMillis, "ms")
AS iso8601

##1. How long did Charlie Chaplin live?
We need to calculate how old Charlie Chaplin was when he died. What code do you use?

In [ ]:
MATCH (p:Person)
WHERE p.name = 'Charlie Chaplin'
RETURN duration.between(p.born,p.died).years

##2. Temporal data stored in graph
How is the born and died properties for a Person node stored in the graph?

In [ ]:
 Date

###Working with dates and times
In the previous lesson, you executed code to create a Test node.

Execute this code to create or update the Test node with these date and datetime values:

In [ ]:
MERGE (x:Test {id: 1})
SET
x.date = date(),
x.datetime = datetime(),
x.timestamp = timestamp(),
x.date1 = date('2022-04-08'),
x.date2 = date('2022-09-20'),
x.datetime1 = datetime('2022-02-02T15:25:33'),
x.datetime2 = datetime('2022-02-02T22:06:12')
RETURN x

###Calculate days between two dates
Write a query to retrieve this Test node and calculate the number of days between date1 and date2.

In [ ]:
// calculate the duration
MATCH (x:Test)
RETURN duration.inDays(x.date1,x.date2).days

##Calculate minutes between two datetime values
Write a query to retrieve this Test node and calculate the number of minutes between datetime1 and datetime2.

In [ ]:
// calculate the duration in minutes
MATCH (x:Test)
RETURN duration.between(x.datetime1,x.datetime2).minutes

#Graph Traversal
As a developer, you must understand what an execution plan is, how to interpret it, and most importantly, how to make it performant. To understand the execution plan, you must understand how a query starts and then how it is processed as the nodes are traversed in the graph.

##Anchor of a query
When the execution plan is created, it determines the set of nodes that will be the starting points for the query. The anchor for a query is often based upon a MATCH clause. The anchor is typically determined by meta-data that is stored in the graph or a filter that is provided inline or in a WHERE clause. The anchor for a query will be based upon the fewest number of nodes that need to be retrieved into memory.

Next, we will look at some examples of how queries are anchored based upon the heuristics used by the graph engine.

In the above query, the Person nodes are the anchor for the query. This is because there are a total of 28,863 nodes in the graph which is what m represents. There are only 19,047 Person nodes so the execution will retrieve fewer nodes if it anchors with the Person nodes.

In [ ]:
PROFILE MATCH (p:Person)-[:ACTED_IN]->(m)
RETURN p.name, m.title LIMIT 100

In the above query the Movie nodes will be the anchor for the query because there are fewer Movie nodes (9,125) than Person nodes (19,047).

In [ ]:
PROFILE MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
RETURN p.name, m.title LIMIT 100

In the above query, a filter is specified **which reduces the number of nodes that will be retrieved for the Person node.** Satisfying the filter is the anchor for the query. **If the Person nodes has an index on name, it only retrieves one record. If there is no index, it needs to scan/filter all Person nodes** for the name property.

In [ ]:
PROFILE MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Eminem'
RETURN p.name, m.title

##Multiple anchors
By default, an anchor set of nodes is determined by the metadata related to the query path and WHERE clauses to filter the query. In some cases you may have more than one set of anchor nodes.

In this query, all p1 nodes are retrieved as well as all p2 nodes. This query has two sets of anchor nodes. It retrieves the anchor nodes before the equality filter is applied. The query planner tries to apply filters as early as possible to reduce cardinality (number of rows).

In [ ]:
PROFILE
MATCH (p1:Person)-[:ACTED_IN]->(m1)
MATCH (m2)<-[:ACTED_IN]-(p2:Person)
WHERE p1.name = 'Tom Hanks'
AND p2.name = 'Meg Ryan'
AND m1 = m2
RETURN m1.title

##Expand to follow paths
After the anchor nodes have been retrieved, the next step if the query specifies a path is to follow the path. The loaded, initial nodes that are part of the anchor set have pointers to relationships that point to nodes on the other end of the relationships.

The goal here is to eliminate paths from the nodes in memory to nodes that will need to be retrieved. This is where specificity in the relationship types is important in your data model.

This query expands to 70 rows because Clint Eastwood has 70 relationships to movies.

In [ ]:
PROFILE MATCH (m:Movie)<--(p:Person)
WHERE p.name = 'Clint Eastwood'
RETURN  m.title

This query expands to fewer rows because Clint Eastwood has fewer DIRECTED relationships to movies.

In [ ]:
PROFILE MATCH (m:Movie)<-[:DIRECTED]-(p:Person)
WHERE p.name = 'Clint Eastwood'
RETURN  m.title

####**In addition, the expansion may lead to the need to inspect properties of the relationship and/or the properties of the Movie node. This inspection means that the nodes are brought into memory and possibly eliminated from the nodes in memory after they have been retrieved.**

####**Cypher queries with multiple MATCH statements may execute differently than what you may expect. You should always profile your queries to understand how the graph is traversed.**

##Basic query traversal
Now let’s learn a little more about how traversal works in Neo4j.

The Eminem Person node is retrieved.

Then the first ACTED_IN relationship is traversed to retrieve the Movie node for 8 Mile.

Then the second ACTED_IN relationship is traversed to retrieve the Movie node for Hip Hop Witch, Da.

The title property is retrieved so that the results can be returned.

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Eminem'
RETURN  m.title AS movies

##Query traversal using multiple patterns
Here is another query that contains two patterns:

For the first pattern in the query, the Eminem Person node is retrieved.

Then the first ACTED_IN relationship is traversed to retrieve the Movie node for 8 Mile.

The second pattern in the query is then used.

Each ACTED_IN relationship to the same 8 Mile movie is traversed to retrieve three co-actors.

If the ACTED_IN relationship has been traversed already, it is not traversed again.

Then the second ACTED_IN relationship is traversed to retrieve the Movie node for Hip Hop Witch, Da.

Each ACTED_IN relationship to the same Hip Hop Witch, Da movie is traversed to retrieve three co-actors.

The title property for the Movie node is retrieved so that the results can be returned.

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie),
(coActors:Person)-[:ACTED_IN]->(m)
WHERE p.name = 'Eminem'
RETURN m.title AS movie ,collect(coActors.name) AS coActors

##Query traversal using multiple MATCH clauses
Here is another query that contains two MATCH clauses:

For the first MATCH clause in the query, the Eminem Person node is retrieved.

Then the first ACTED_IN relationship is traversed to retrieve the Movie node for 8 Mile.

The second MATCH clause in the query is then executed.

Each ACTED_IN relationship to the same 8 Mile movie is traversed to retrieve all actors, including the relationship to the Eminem node.

Then the query returns back to the first MATCH clause to traverse the ACTED_IN relationship to the Hip Hop Witch, Da movie.

The second MATCH clause in the query is then executed.

Each ACTED_IN relationship to the same Hip Hop Witch, Da movie is traversed to retrieve all actors.

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Eminem'
MATCH (allActors:Person)-[:ACTED_IN]->(m)
RETURN m.title AS movie, collect(allActors.name) AS allActors

##Avoiding labels for better performance
Another graph optimization that you can take advantage of is to **reduce labels used in your query patterns.** Having a label for the anchor nodes in a pattern is good:


**The Person label for the anchor node retrieval is good here, but the label for the other side of the pattern is unnecessary. Having the label on the non-anchor node forces a label check, which is really not necessary.**

In [ ]:
PROFILE MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
RETURN m.title AS movie

#Cypher version: 5, planner: COST, runtime: PIPELINED. 195 total db hits in 24 ms.

Here is a more performant way to do this query:

In [ ]:
PROFILE MATCH (p:Person)-[:ACTED_IN]->(m)
WHERE p.name = 'Tom Hanks'
RETURN m.title AS movie

#Cypher version: 5, planner: COST, runtime: PIPELINED. 119 total db hits in 2 ms.

##Returning paths
In Neo4j Browser, when you return nodes, **by default the relationships are visualized.** For example, this query is visualized with its associated paths that were traversed for the query:

The visualization includes one Person node that is connected to four Movie nodes using five relationships. The objects that are returned (table view), are five rows where the Person node values are in each row and the one Movie node is repeated. There is no relationship information returned.



In [ ]:
MATCH (person:Person)-[]->(movie)
WHERE person.name = 'Walt Disney'
RETURN person, movie

You can return paths in your query as follows:

This query returns 5 paths. If you view the objects (table view in Neo4j Browser), you will see that each row returned represents the Person node, the Movie node, and the relationship.

In [ ]:
MATCH p = ((person:Person)-[]->(movie))
WHERE person.name = 'Walt Disney'
RETURN p

###In some applications, it may be useful to work with path objects. Cypher has some useful functions that can be used to analyze paths:

1. length(p) returns the length of a path.

2. nodes(p) returns a list containing the nodes for a path.

3. relationships(p) returns a list containing the relationships for a path.

#1. Best query performance
We want to return a list of names of reviewers who rated the movie, Toy Story. What query will perform best?

###Solution

The correct answer is: MATCH (movie:Movie)←[:RATED]-(reviewer) will perform the best because it uses a label for the anchor node, Movie and specifies the relationship type.

You need not specify a label for non-anchor nodes in the pattern.

Specifying the relationship type in a pattern will always yield better performance.

In [ ]:
MATCH (movie:Movie)←[:RATED]-(reviewer)
WHERE movie.title = 'Toy Story'
RETURN reviewer.name

##2. Traversing the graph
What term best describes the traversal behavior during a query?

In [ ]:
 depth-first

##Traversing the Graph

How many relationships are traversed to return the result? Ans:16

m.title

collect(coActors.name)

------------
"Money Train"

["Wesley Snipes", "Woody Harrelson", "Jennifer Lopez"]

---------------

"Paul Williams Still Alive"

["Paul Williams", "Warren Beatty", "Karen Carpenter"]

------------------------

"In Cold Blood"

["John Forsythe", "Paul Stewart", "Scott Wilson"]

-----------------------------

"Electra Glide in Blue"

["Billy Green Bush", "Jeannine Riley", "Mitchell Ryan"]


###Hint

The query finds each movie that Robert Blake acted in. How many ACTED_IN traversals is that?

For each movie, it uses a second MATCH to find all actors to acted in that movie. Because there are two MATCH clauses, it traverses the ACTED_IN relationship from Robert Blake twice.

How many ACTED_IN relationships are traversed?

In [ ]:
MATCH (p:Person {name: 'Robert Blake'})-[:ACTED_IN]->(m:Movie),
 (coActors:Person)-[:ACTED_IN]->(m)
RETURN m.title, collect(coActors.name)

##Relationships Traversed

How many relationships are traversed to return the result?
Answer: 20

m.title

collect(allActors.name)

-----------------------

"Money Train"

["Wesley Snipes", "Robert Blake", "Woody Harrelson", "Jennifer Lopez"]

--------------------

"Paul Williams Still Alive"

["Paul Williams", "Robert Blake", "Warren Beatty", "Karen Carpenter"]

-------------------------

"In Cold Blood"

["John Forsythe", "Paul Stewart", "Robert Blake", "Scott Wilson"]

--------------

"Electra Glide in Blue"

["Billy Green Bush", "Robert Blake", "Jeannine Riley", "Mitchell Ryan"]


In [ ]:
MATCH (p:Person {name: 'Robert Blake'})-[:ACTED_IN]->(m:Movie)
MATCH (allActors:Person)-[:ACTED_IN]->(m)
RETURN m.title, collect(allActors.name)

#Varying Length Traversal

Any graph that represents social networking, hierarchies, transport, flow, or dependency networks will most likely have multiple paths of varying lengths. Think of the connected relationship in LinkedIn and how connections are made by people connected to more people.

Here are two use cases for this type of traversal:

1. Finding the shortest path between two nodes.

2. Finding out how "close" nodes are to each other in the graph.

**In Neo4j uniqueness of relationships is always adhered to. That is, there will never be two relationships of the same type and direction between two nodes. This enables Neo4j to avoid cycles or infinite loops in graph traversal.**

##Shortest path
Cypher has a built-in function that returns the **shortest path between any two nodes, if one exists.**

This query calculates and returns the shortest path between the Eminem node and the Charlton Heston node in the graph. Because nothing is specified for the relationship type, it finds the shortest path regardless of the relationship type. That is, Eminem is four hops away from Charlton Heston using the ACTED_IN and RATED relationships.

In [ ]:
MATCH p = shortestPath((p1:Person)-[*]-(p2:Person))
WHERE p1.name = "Eminem"
AND p2.name = "Charlton Heston"
RETURN  p

You can also limit the relationship types to a specific relationship:

This query **returns a longer path because the relationship must be ACTED_IN.**

In [ ]:
MATCH p = shortestPath((p1:Person)-[:ACTED_IN*]-(p2:Person))
WHERE p1.name = "Eminem"
AND p2.name = "Charlton Heston"
RETURN  p

##**For shortestPath() and allShortestPaths() you can provide an upper bound on the length of the path(s), but not a lower bound.**

##Varying length traversal
Suppose you want to retrieve all Person nodes that are exactly two hops away from Eminem using the ACTED_IN relationship.

The Eminem Person node is retrieved.

Then the two ACTED_IN relationship are traversed through the Movie node for 8 Mile to return the three Person nodes.

Then the two ACTED_IN relationships are traversed through the Movie node for Hip Hop Witch, Da to return the three Person nodes.

In [ ]:
MATCH (p:Person {name: 'Eminem'})-[:ACTED_IN*2]-(others:Person)
RETURN  others.name

##Four hops away
Suppose you want to retrieve all Person nodes that are exactly four hops away from Eminem using the ACTED_IN relationship.

The Eminem Person node is retrieved.

Then the ACTED_IN relationships are traversed through the Movie node where Brittany Murphy for 8 Mile is retrieved and Little Black Book to return the two Person nodes.

Then the four ACTED_IN relationships are traversed through the Movie node for 8 Mile and The Prophecy II to return the two Person nodes.

Only Person nodes that are exactly 4 hops from Eminem are returned.

In [ ]:
MATCH (p:Person {name: 'Eminem'})-[:ACTED_IN*4]-(others:Person)
RETURN  others.name

##1. What actors are up to 6 hops away?
We want to return a list of actors that are up to 6 hops away from Tom Hanks.

How do you specify this?


###Solution

The correct answer is: MATCH (p:Person)-[:ACTED_IN*1..6]-(others:Person) which returns Person nodes that are up to 6 hops away.

MATCH (p:Person)-[:ACTED_IN*6]-(others:Person) returns all Person nodes that are exactly 6 hops away.

The other choices are not valid Cypher.

In [ ]:
MATCH (p:Person)-[:ACTED_IN*1..6]-(others:Person)
WHERE p.name = 'Tom Hanks'
RETURN  others.name

##2. Finding the shortest path between 2 nodes
What Cypher function returns the shortest path between 2 nodes?

In [ ]:
shortestPath()

####Write and execute the query to return the names of actors that are 2 hops away from Robert Blake using the ACTED_IN relationship.

How many actors are returned in the result?

In [ ]:
MATCH (p:Person {name: 'Robert Blake'})-[:ACTED_IN*2]-(others:Person)
RETURN  others.name

####Write and execute the query to return the distinct names of actors that are 4 hops away from Robert Blake using the ACTED_IN relationship.

How many actor names are returned in the result?

In [ ]:
MATCH (p:Person {name: 'Robert Blake'})-[:ACTED_IN*4]-(others:Person)
RETURN  DISTINCT others.name

####Write and execute the query to return the unique names of actors that up to 4 hops away from Robert Blake using the ACTED_IN relationship.

How many unique actors are returned in the result?

In [ ]:
MATCH (p:Person {name: 'Robert Blake'})-[:ACTED_IN*1..4]-(others:Person)
RETURN  DISTINCT others.name

##Scoping variables for a query
You have already learned **how you can specify a variable for a node or relationship in a query:**

In this query the variable p is used to test each Person node against the value Tom Hanks. The variable m is used to return the movie titles.

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
RETURN m.title AS movies

###You can define and initialize variables to be used in the query with a WITH clause.

Before the MATCH clause, we define a variable, actorName to have a value of Tom Hanks. The variable, actorName is in the scope of the query, so it can be used like a parameter. The query itself can be reused with a different value for actorName.

In [ ]:
WITH 'Tom Hanks' AS actorName
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = actorName
RETURN m.title AS movies

####Using WITH to redefine scope

For this query, mt and actorName are within scope of the MATCH clause that also uses the WHERE clause. It retrieves the Person node, then all the movies that Tom Hanks acted in, then it filters and returns the movies that contain mt.

In [ ]:
WITH  'toy story' AS mt, 'Tom Hanks' AS actorName
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = actorName
AND toLower(m.title) CONTAINS mt
RETURN m.title AS movies

###Now, let’s look at this query:

The variables mt, and actorName are available to the MATCH clause and the WHERE clause just like the previous query. What is different here, however, is that we must add the m to the second WITH clause so that the node can be used to return the title of the node. A WITH clause is used to define or redefine the scope of variables. Because we want to redefine what is used for the WHERE clause, we add a new WITH clause. This creates a new scope for the remainder of the query so that m and movieTitle can be used to return values. If you were to remove the m in the second WITH clause, the query would not compile.

Notice also that you can define expressions using WITH. When you define an expression (for example, toLower(m.title), you must specify an alias defined with the AS keyword.

In [ ]:
WITH  'toy story' AS mt, 'Tom Hanks' AS actorName
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WITH m, toLower(m.title) AS movieTitle
WHERE p.name = actorName
AND movieTitle CONTAINS mt
RETURN m.title AS movies, movieTitle

###Limiting results
Suppose we have this query where we want to return only two rows:

In [ ]:
WITH  'Tom Hanks' AS theActor
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = theActor
RETURN m.title AS movies LIMIT 2

Another way to write this query is:



With this query, two Movie nodes are retrieved. What is different here is that you can use WITH to limit how many m nodes are used later in the query. Passing nodes on to the next MATCH clause is called pipelining that you will learn about in the next lesson.


In [ ]:
WITH  'Tom Hanks' AS theActor
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = theActor
WITH m  LIMIT 2
// possibly do more with the two m nodes
RETURN m.title AS movies

##Ordering results
If you are limiting the nodes to process further on in the query or for the RETURN clause, you can also order them:

In [ ]:
WITH  'Tom Hanks' AS theActor
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = theActor
WITH m ORDER BY m.year LIMIT 5
// possibly do more with the five m nodes in a particular order
RETURN m.title AS movies, m.year AS yearReleased

##Using map projections in a WITH clause
Here is another example where we want to customize the properties of nodes that are returned. Using a map projection, you can specify which properties are returned. This type of customization of nodes returned is very useful when you are integrating with an application.


This query returns a subset of the data in a Movie node. It returns the top four rated movies. Because we have specified a limit of four, only 4 objects with the specified properties are added to the list. This type of data returned is commonly used by GraphQL and JavaScript applications.

In [ ]:
MATCH (n:Movie)
WHERE n.imdbRating IS NOT NULL
AND n.poster IS NOT NULL
WITH n {
  .title,
  .year,
  .languages,
  .plot,
  .poster,
  .imdbRating,
  directors: [ (n)<-[:DIRECTED]-(d) | d { tmdbId:d.imdbId, .name } ]
}
ORDER BY n.imdbRating DESC LIMIT 4
RETURN collect(n)

##1. Scoping variables
Here is a query to return the name of the actor (Clint Eastwood) and all the movies that he acted in that contain the string 'high'. How do you complete this query so it can return the desired results?

###Solution

The correct answer is: WITH p, m, toLower(m.title) AS movieTitle. movieTitle is created during the query and is passed down for use in the WHERE clause. In order to use the variables p and m, they must also be included.

If you do not include both p and m in the WITH clause (which re-scopes variables) these variables cannot be used later in the query and in the RETURN clause.

In [ ]:
WITH  'Clint Eastwood' AS a, 'high' AS t
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WITH p, m, toLower(m.title) AS movieTitle
WHERE p.name = a
AND movieTitle CONTAINS t
RETURN p.name AS actor, m.title AS movie

##2. Cypher scoping
What Cypher keyword is used to redefine the scope of variables in a query?

In [ ]:
WITH

###What is the highest revenue movie for Tom Hanks?
What movie had the highest revenue? (Note, the answer is case-sensitive)?

In [ ]:
WITH  'Tom Hanks' AS theActor
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = theActor
AND m.revenue IS NOT NULL
WITH m ORDER BY m.revenue DESC LIMIT 1
RETURN m.revenue AS revenue, m.title AS title

###Using WITH for map projection to limit results
Here is a query, a version of which you saw in the previous lesson:

Modify this query by adding a WITH clause that customizes the data returned for each Movie node to include:

title

imdbRating

List of actor names

List of Genre names

Then answer this question:

Which actor?
What actor acted in more than one of these top four movies? (Name is case-sensitive)

In [ ]:
MATCH (n:Movie)
WHERE n.imdbRating IS NOT NULL AND n.poster IS NOT NULL
WITH n {
  .title,
  .imdbRating,
  actors: [ (n)<-[:ACTED_IN]-(p) | p { tmdbId:p.imdbId, .name } ],
  genres: [ (n)-[:IN_GENRE]->(g) | g {.name}]
}
ORDER BY n.imdbRating DESC
LIMIT 4
RETURN collect(n)

How many unique genres are represented in these top four movies?

In [ ]:
MATCH (n:Movie)
WHERE n.imdbRating IS NOT NULL AND n.poster IS NOT NULL
WITH n {
  .title,
  .imdbRating,
  actors: [ (n)<-[:ACTED_IN]-(p) | p { tmdbId:p.imdbId, .name } ],
  genres: [ (n)-[:IN_GENRE]->(g) | g {.name}]
}
ORDER BY n.imdbRating DESC
LIMIT 4
RETURN collect(n)

##What does pipelining mean?
You have learned that you can use the WITH clause to redefine the scope for the query:

In [ ]:
WITH  'Tom Hanks' AS theActor
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = theActor
WITH m  LIMIT 5
// possibly do more with the five m nodes
RETURN m.title AS movies

This query limits the query to retrieve five nodes.

You can do something with the five nodes by adding another MATCH clause:

The **five Movie nodes are used for the second MATCH clause and five rows are returned.** The **WITH clause** enables us to **pipeline the results of the first query into the second query.** With this simple query, m need not be redefined or scoped.

Next you will learn how pipelining is really useful.

In [ ]:
WITH  'Tom Hanks' AS theActor
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = theActor
WITH m  LIMIT 5
MATCH (d:Person)-[:DIRECTED]->(m)
RETURN d.name AS director,
m.title AS movies

##Using WITH for aggregation
A very common use of WITH is to aggregate data so that intermediate results can be used to create the final returned data or to pass the intermediate results to the next part of the query.

In this query we are counting the number of movies that share the same Genre node. We use count() to count the number of rows and sum() to total the imdbRating for each movie for the Genre. These values are calculated as part of the aggregation and then used to return the data. In the WITH clause we pass on only the values we need to return a row for the Genre.

In [ ]:
MATCH (:Movie {title: 'Toy Story'})-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(m)
WHERE m.imdbRating IS NOT NULL
WITH g.name AS genre,
count(m) AS moviesInCommon,
sum(m.imdbRating) AS total
RETURN genre, moviesInCommon,
total/moviesInCommon AS score
ORDER By score DESC

Here is another example that shows aggregation and pipelining:

For this query, we first calculate the average rating for all movies that Misty Williams rated. Then we use this calculated value, average as a test for the second MATCH.

In [ ]:
MATCH (u:User {name: "Misty Williams"})-[r:RATED]->(:Movie)
WITH u, avg(r.rating) AS average
MATCH (u)-[r:RATED]->(m:Movie)
WHERE r.rating > average
RETURN average , m.title AS movie,
r.rating as rating
ORDER BY rating DESC

##Using WITH for collecting
Another common use for the WITH clause is to collect results into a list that will be returned:

This query collects the names of actors who acted in the movies containing the string 'New York'. This aggregation collects the names and totals the number of actors.

In [ ]:
MATCH (m:Movie)--(a:Actor)
WHERE m.title CONTAINS 'New York'
WITH m, collect (a.name) AS actors,
count(*) AS numActors
RETURN m.title AS movieTitle, actors
ORDER BY numActors DESC

Here is another example where we perform a 2-step aggregation for collecting a list of maps:

In [ ]:
MATCH (m:Movie)<-[:ACTED_IN]-(a:Actor)
WHERE m.title CONTAINS 'New York'
WITH m, collect (a.name) AS actors,
count(*) AS numActors
ORDER BY numActors DESC
RETURN collect(m { .title, actors, numActors }) AS movies

##Using LIMIT early
A best practice is to execute queries that minimize the number of rows processed in the query. One way to do that is to limit early in the query. This also helps in reducing the number of properties loaded from the database too early.

For example, this query limits during the aggregation:

In [ ]:
PROFILE MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.born.year = 1980
WITH p,
collect(m.title) AS movies LIMIT 3
RETURN p.name AS actor, movies

A better way to do this query is to limit early. This query will perform slightly better because LIMIT is moved up in the query:

In [ ]:
PROFILE MATCH (p:Actor)
WHERE p.born.year = 1980
WITH p  LIMIT 3
MATCH (p)-[:ACTED_IN]->(m:Movie)
WITH p, collect(m.title) AS movies
RETURN p.name AS actor,  movies

Use DISTINCT when necessary
Here is an example of a query that retrieves three actors, then collects the names of the genres for the movies that actor acted in.

In [ ]:
MATCH (p:Actor)
WHERE p.born.year = 1980
WITH p  LIMIT 3
MATCH (p)-[:ACTED_IN]->(m:Movie)-[:IN_GENRE]->(g:Genre)
WITH p, collect(g.name) AS genres
RETURN p.name AS actor, genres

Notice that for this query, the collected genre names are repeated. You would want to ensure that they are not duplicated as follows:

In [ ]:
MATCH (p:Actor)
WHERE p.born.year = 1980
WITH p  LIMIT 3
MATCH (p)-[:ACTED_IN]->(m:Movie)-[:IN_GENRE]->(g:Genre)
WITH p, collect(DISTINCT g.name) AS genres
RETURN p.name AS actor, genres

###Limiting results
Here is a query to return the names of Directors who directed movies that Keanu Reeves acted in. How do you specify that you want to limit the number of rows returned to 3?

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Keanu Reeves'
WITH m LIMIT 3
MATCH (d:Person)-[:DIRECTED]->(m)
RETURN collect(d.name) AS directors,
m.title AS movies

##2. Cypher for aggregating
What Cypher functions/keywords are used to aggregate intermediate results?

In [ ]:
count(), collect()

##Using WITH to pass on intermediate results
Write and execute a query to determine the highest average rating by a user for a Tom Hanks Movie. Use avg(r.rating) to aggregate the rating values for all movies that Tom Hanks acted in, where you use the pattern (m:Movie)←[r:RATED]-(:User).

Then answer this question:

What Tom Hanks movie had the highest average rating greater than 4?

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)<-[r:RATED]-(:User)
WHERE p.name = 'Tom Hanks'
WITH m, avg(r.rating) AS avgRating
WHERE avgRating > 4
RETURN m.title AS Movie, avgRating AS `AverageRating`
ORDER BY avgRating DESC

What is the highest average rating for a movie acted in by Tom Hanks?
Based upon the query you just executed, enter the highest average rating.

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)<-[r:RATED]-(:User)
WHERE p.name = 'Tom Hanks'
WITH m, avg(r.rating) AS avgRating
WHERE avgRating > 0
RETURN m.title AS Movie, avgRating AS `AverageRating`
ORDER BY avgRating DESC

##Using UNWIND
You have learned that in a query, you can create lists of nodes, strings, or numbers using collect(). Sometimes it is useful to collect elements as intermediate results that are passed on to a later part of a query. For example, the graph you are working with contains languages and countries lists for each Movie node. If you wanted to refactor the graph to create a Language node and associate it with any Movie node that had that particular language in its languages list, you could unwind the list to access each element in the list.

**UNWIND returns a row for each element of a list.**

In this query:

1. A Movie is retrieved that Tom Hanks acted in.

2. The languages property, which is a list is unwound and each value is referenced as lang.

3. The rows returned will be the movie title and the languages property repeated for multiple rows and the lang value.

In this code, you could imagine that you could create a Language node with a name property which is lang.

In [ ]:
MATCH (m:Movie)-[:ACTED_IN]-(a:Actor)
WHERE a.name = 'Tom Hanks'
UNWIND m.languages AS lang
RETURN m.title AS movie,
m.languages AS languages,
lang AS language

Let’s now look at a more useful example.

This query:

1. Retrieves all Movie nodes.

2. For each Movie node, it unwinds the languages list to create a list called lang. Notice that we use the trim() function to ensure there are no extraneous whitespace characters in the language name.

3. Then we use the element of the list to find all Movies that use that language.

4. Finally, we return a row that contains each language name and the list of up to 10 movie titles for that language.

In [ ]:
MATCH (m:Movie)
UNWIND m.languages AS lang
WITH m, trim(lang) AS language
// this automatically, makes the language distinct because it's a grouping key
WITH language, collect(m.title) AS movies
RETURN language, movies[0..10]

#Hint

**The UNWIND keyword in Cypher is used to transform a collection into a sequence of rows for further processing.**

##1. Using UNWIND
Here is a query to return the title of a movie and language pair for any Tom Hanks movie that has exactly two languages associated with it. That is each movie will have two rows, the title is repeated and then each language for that title. How do you complete this query to create a lang value that is an element of the languages list?

In [ ]:
MATCH (m:Movie)-[:ACTED_IN]-(a:Actor)
WHERE a.name = 'Tom Hanks'
AND size(m.languages) = 2
UNWIND m.languages AS lang
RETURN m.title AS movie,lang AS languages

##2. Using UNWIND
What type of data can UNWIND be used for?

In [ ]:
The answers are List of strings and List of numerics.

##Using UNWIND pass on intermediate results
You have already seen this query:

Hint

You will unwind by countries, rather than languages.

After you have collected the movies for each language as a list, return the size of the list for each country.

How many movies released in the UK are in the graph?

In [ ]:
MATCH (m:Movie)
UNWIND m.countries AS country
WITH m, trim(country) AS trimmedCountry
// this automatically, makes the trimmedCountry distinct because it's a grouping key
WITH trimmedCountry, collect(m.title) AS movies
RETURN trimmedCountry, size(movies)

##Using UNWIND pass on intermediate results
Using the query you just created, answer this question:

Movies released in Taiwan?
How many movies released in Taiwan are in the graph?

In [ ]:
MATCH (m:Movie)
UNWIND m.countries AS country
WITH m, trim(country) AS trimmedCountry
// this automatically, makes the trimmedCountry distinct because it's a grouping key
WITH trimmedCountry, collect(m.title) AS movies
RETURN trimmedCountry, size(movies)

##What is a subquery?
When you execute a series of MATCH clauses, all nodes and relationships retrieved are in memory. If the memory requirements for a set of MATCH clauses exceed the VM configured, the query will fail.

A subquery is a set of Cypher statements that execute within their own scope. A subquery is typically called from an outer enclosing query. Using a subquery, you can limit the number of rows that need to be processed.

Here are some important things to know about a subquery:

A subquery returns values referred to by the variables in the RETURN clause.

A subquery cannot return variables with the same name used in the enclosing query.

You must explicitly pass in variables from the enclosing query to a subquery.

##Performing subqueries with CALL
In a CALL clause, you specify a query that can return data from the graph or derived from the graph. A set of nodes returned in the CALL clause can be used by the enclosing query.

The subquery is demarcated by the {}s here. The subquery returns 10 Movie nodes of movies released in 2000 with the highest imdbRating. Then with these 10 movies, it determines the average rating for each of these 10 movies.

In [ ]:
CALL {
   MATCH (m:Movie) WHERE m.year = 2000
   RETURN m ORDER BY m.imdbRating DESC LIMIT 10
}
MATCH  (:User)-[r:RATED]->(m)
RETURN m.title, avg(r.rating)

##Passing variables into a subquery
Here is an example where the subquery is executed after the initial query and the enclosing query passes a variable, m into the subquery.

In this query:

The first MATCH returns a row for every movie, m in the graph.

It passes the Movie node, m to the subquery.

Then within the subquery, the query executes to find all users who gave that movie a rating of 5 and counts them.

The subquery returns the count.

Back in the enclosing query, the title is returned, and the count of the number of rows returned from the subquery.

**Using subqueries enables you to reduce the number of rows processed in a query.**

In [ ]:
MATCH (m:Movie)
CALL {
    WITH m
    MATCH (m)<-[r:RATED]-(u:User)
     WHERE r.rating = 5
    RETURN count(u) AS numReviews
}
RETURN m.title, numReviews
ORDER BY numReviews DESC

##Combining query results with UNION
As your queries become more complex, you may need to combine the results of multiple queries. You can do so with UNION. With UNION, the queries you are combining must return the same number of properties or data

The first query returns an object with a type property of "movies" and a theMovies property that is a list of movies. It returns this object as a variable named Data

The second query returns an object with a type property of "actors" and a theActors property that is a list of actor names. It returns this object as a variable named Data

Because both queries return a variable named Data, we can combine the results using UNION ALL.

**UNION ALL returns all results which is more efficient on memory but can lead to duplicates. UNION returns distinct results.**

In [ ]:
MATCH (m:Movie) WHERE m.year = 2000
RETURN {type:"movies", theMovies: collect(m.title)} AS data
UNION ALL
MATCH (a:Actor) WHERE a.born.year > 2000
RETURN { type:"actors", theActors: collect(DISTINCT a.name)} AS data

##Using UNION with subqueries
Results of a UNION cannot be directly post-processed. But if you wrap a UNION in a subquery, you can then further process the results.

Here is an example that uses UNION within a subquery:

This query:

100 Person nodes are retrieved and passed to the subquery.

If that Person acted in the movie, its title with the Actor suffix is returned.

The second part of the subquery does the same for the DIRECTED relationships.

The work results are combined and collected.

The result is the name of the person and their Actor or Director titles.

In [ ]:
MATCH (p:Person)
WITH p LIMIT 100
CALL {
  WITH p
  OPTIONAL MATCH (p)-[:ACTED_IN]->(m:Movie)
  RETURN m.title + ": " + "Actor" AS work
UNION
  WITH p
  OPTIONAL MATCH (p)-[:DIRECTED]->(m:Movie)
  RETURN m.title+ ": " +  "Director" AS work
}
RETURN p.name, collect(work)

##1. Using a subquery
Here is a query that has a subquery. The enclosing query finds all User nodes. The subquery finds all movies that this user rated with 5 and return them. How do you complete this query so it can return the desired results?

In [ ]:
cypher
MATCH (u:User)
CALL {
WITH u

    MATCH (m:Movie)<-[r:RATED]-(u)
     WHERE r.rating = 5
    RETURN m
}
RETURN m.title, count(m) AS numReviews
ORDER BY numReviews DESC

##2. Combining query results
What Cypher keyword is used to combine the results of multiple MATCH clauses into a single return stream?

In [ ]:
 UNION ALL

##Add a subquery
Top French movie

Write and execute the subquery to return the movie nodes that have a countries list element of 'France'.

You must pass in the g variable to the subquery.

You must test if the movie node for that genre was released in France.

The subquery must return the count of the movie nodes, numMovies for each Genre passed in.

How many movies were in the largest Genre category for movies released in France? (enter a number)

In [ ]:
MATCH (g:Genre)
CALL { WITH g
MATCH (g)<-[:IN_GENRE]-(m) WHERE 'France' IN m.countries
RETURN count(m) AS numMovies
}
RETURN g.name AS Genre, numMovies ORDER BY numMovies DESC

##Combining actors and directors data
Actors and Directors from 2015
Here is a query that returns actor information for the year 2015:

Now add another query to this code to return the directors for 2015. Use UNION ALL to combine results. The second query will return the string "Director" as Type.

How many rows are returned?

In [ ]:
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
WHERE m.year = 2015
RETURN "Actor" AS type,
p.name AS workedAs,
collect(m.title) AS movies
UNION ALL
MATCH (m:Movie)<-[:DIRECTED]-(p:Person)
WHERE m.year = 2015
RETURN "Director" AS type,
p.name AS workedAs,
collect(m.title) AS movies

##1. What parameters to use?
Suppose we have set the following parameters:


:params {actorName: 'Tom Cruise', movieName: 'Top Gun', l:2}

Here is a query to return the names of the actors in a particular movie that is parameterized. The number of results returned is also parameterized. How would you complete this query in a Neo4j Browser session?

In [ ]:
:params {actorName: 'Tom Cruise', movieName: 'Top Gun', l:2}

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.title = $movieName RETURN p.name LIMIT $l

##2. Parameters for the session?
What command in Neo4j Browser returns all parameters set for the session?

In [ ]:
:params

##3. Setting Integers
Which command would you use to ensure that the value of the myNumber parameter is cast as an integer?

In [ ]:
 :param myNumber ⇒ 10

##Setting Parameters
You have been given the following query which finds all users with a name beginning with the string value supplied in the $name parameter.

###Hint

The :param command allows you to set a parameter which is persisted as long as the browser window is open. The format is as follows:


##**:param key: "value"**

What command would you run in Neo4j Browser to set the $name parameter to Tom? (use double-quotes for the value of the parameter)

In [ ]:
:param name: "Tom"

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name STARTS WITH $name
RETURN p.name AS actor,
m.title AS title

#####In this challenge, you will need to execute the following query to find all movies acted in by an Actor with a name starting with Tom, and that also have the value UK within the countries array on the movie.

In [ ]:
#1. Set the name Parameter
#Using the same format as the previous lesson, set the name parameter to Tom.
:param name: "Tom"

In [ ]:
#2. Set the country Parameter
#Set the country parameter to UK.
:param country: "UK"

In [ ]:
#3. Execute the Query
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name STARTS WITH $name
AND $country IN m.countries
RETURN p.name AS actor,
m.title AS title

##Application Examples Using Parameters
Although each language has its own distinct syntax, the process for each language is similar. Parameters are sent as an additional argument when a Cypher statement is run through the driver, and are referenced in the same way as covered in this module, by applying the **$ prefix.**

Example: Java
Here is a code snippet that illustrates the use of parameters in **Java:**

In this example, the parameters are passed to the second argument of the tx.run method using the static parameters function provided by the org.neo4j.driver.Values class.

In [ ]:
try (var session = driver.session()) {
  Result res = session.readTransaction(tx -> tx.run("""
    MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
    WHERE m.title = $title
    RETURN p
    LIMIT 10
  """, Values.parameters("title", "Toy Story")));
}

##Example: JavaScript
Here is a code snippet that illustrates the use of parameters in JavaScript:

In this example, the parameters are passed to the second argument of the tx.run method as a JavaScript object.

In [ ]:
const session = driver.session()
const res = await session.readTransaction(tx =>
  tx.run(`
    MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
    WHERE m.title = $title
    RETURN p
    LIMIT 10
  `,
  { title: 'Toy Story'})
)

##Example: Python
Here is a code snippet that illustrates the use of parameters in Python:

In Python, Cypher parameters are passed as named parameters to the tx.run method. In this example, title has been passed as a named parameter.

In [ ]:
def get_actors(tx, movieTitle): # (1)
  result = tx.run("""
    MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
    WHERE m.title = $title
    RETURN p
  """, title=movieTitle)

  # Access the `p` value from each record
  return [ record["p"] for record in result ]

with driver.session() as session:
    result = session.read_transaction(get_actors, movieTitle="Toy Story")

####What character is used to prefix parameters in a Cypher statement?

In [ ]:
$

##Setting a parameter
You can set values for Cypher parameters that will be in effect during your Neo4j Browser session.

You can set the value of a single parameter in the query editor pane as shown in this example where the value Tom Hanks is set for the parameter actorName:

In [ ]:
:param actorName: 'Tom Hanks'

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = $actorName
RETURN m.released AS releaseDate,
m.title AS title
ORDER BY m.released DESC

##Setting a parameter
You can set values for Cypher parameters that will be in effect during your Neo4j Browser session.

You can set the value of a single parameter in the query editor pane as shown in this example where the value Tom Hanks is set for the parameter actorName:

In [ ]:
:param actorName: 'Tom Hanks'

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = $actorName
RETURN m.released AS releaseDate,
m.title AS title
ORDER BY m.released DESC

##Setting Integers
Special consideration should be made when setting integer values in a Neo4j Browser session. Due to a discrepancy between integers in JavaScript and the Neo4j type system, any integers are converted to floating point values when the parameter is set. This is designed to avoid any data loss on large numbers.

For example, if you run the following code to set the number parameter using colon (:) operator, the number will be converted from 10 to 10.0.

In [ ]:
:param number: 10

In [ ]:
:param number=> 10

##Setting multiple parameters
You can also use the JSON-style syntax to set all of the parameters in your Neo4j Browser session. The values you can specify in this object are numbers, strings, and booleans. In this example we set two parameters for our session:

In [ ]:
:params {actorName: 'Tom Cruise', movieName: 'Top Gun'}

If you have set multiple parameters, you can use the ⇒ operator as seen before to add more parameters to a set of parameters.

In [ ]:
:param number=> 10

##Using multiple parameters
Here is a different query that uses both parameters:

In [ ]:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = $actorName
AND m.title = $movieName
RETURN p, m

##Viewing parameters
If you want to view the current parameters and their values, simply type:

In [ ]:
:params

##Removing parameters
If you want to remove an existing parameter from your session, you do so by using the JSON-style syntax and exclude the parameter for your session.

If you want to clear all parameters, you can simply type:

In [ ]:
:params {}